In [ ]:
import numpy as np
import keras as ks
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras.constraints import maxnorm
from keras.layers import Dropout
from keras.optimizers import Adamax
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor




nod = 1058          #===========no. of data points============#
k1 = np.array([0.1903934, 0.3220935, 1.559453 ])
k2byk1 = np.arange(0.50,1.05,0.05) 		#======Ratio k2byk1========#

cosalpha = np.arange(0.50,1.00,0.01)	#======cosine of the angle between the k2 and k1 arms =======#

k2byk1 = k2byk1.reshape(11,1)

costheta = -cosalpha

B_02 = np.loadtxt('Bk_02f')
params_02 = np.loadtxt('params_02f')
x_H1 = np.loadtxt('x_H1_02f')

#path = '/home/ht/Desktop/bispec_data/'
#===============Reading the data files===================#
Omega_bh2 = 0.0224
Omega_mh2 = 0.1424                                      #==============present day values Omega_mh2,Omega_bh2====#
z = 9.210
#================================================================================================================#

dT_b = 27.0 * x_H1 * (Omega_bh2 / 0.023) * np.power(((0.15 * (1. + z)) / (10. * Omega_mh2)), 0.50)
dT_b.reshape(len(dT_b))
dt = (dT_b**3)

B_022 = np.zeros(shape=(len(dt),550))

for i in range(len(dt)):
    B_022[i] = B_02[i]*dt[i]

    
B_02 = B_022    

B_02 = B_02/100.
  

Bk_train , Bk_test , params_train ,params_test = train_test_split(B_02,params_02,test_size = 0.05,shuffle = True)
init_mode='uniform'
def create_model(neurons1 =1,neurons2 =1):
    model = Sequential()
    model.add(Dense(80, input_shape=[3,], activation='elu',kernel_initializer=init_mode,))
    model.add(Dense(320,activation='elu',kernel_initializer=init_mode))
    model.add(Dense(460,activation='elu',kernel_initializer=init_mode))
    model.add(Dense(560,activation='elu',kernel_initializer=init_mode))
    model.add(Dense(260,activation='elu',kernel_initializer=init_mode))
    model.add(Dense(100,activation='elu',kernel_initializer=init_mode))
    model.add(Dense(neurons1,activation='elu',kernel_initializer=init_mode))
    model.add(Dense(neurons2,activation='elu',kernel_initializer=init_mode))
    #model.add(Dropout(dropout_rate))
    model.add(Dense(550, activation='linear'))
    # Compile model
    optimizer = Adamax(lr=0.0001, beta_1=0.2, beta_2=0.088)
    model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
    return model
# fix random seed for reproducibility

'''
    model.add(Dense(neurons5,activation='elu'))
    model.add(Dense(neurons6,activation='elu'))
'''   
seed = 7
np.random.seed(seed)

model = KerasRegressor(build_fn=create_model,epochs=1000,batch_size=50,verbose=0)
neurons1 = [10,40,60,80,100,300,320,340,360,380,400]
neurons2 = [300,320,340,360,380,400]
'''
neurons3 = [10,40,100,160,260,360,460,560]
neurons4 = [10,40,100,160,260,360,460,560]

neurons5 = [10,40,100,160,260,360,460,560]
neurons6 = [10,40,100,160,260,360,460,560]
learn_rate = [0.0001,0.001, 0.01, 0.1, 0.2, 0.3]
beta_1 = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
beta_2 = [0.0, 0.02, 0.004, 0.06, 0.088, 0.0999]


init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
'''
param_grid = dict(neurons1 = neurons1,neurons2=neurons2)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(params_train, Bk_train)


print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

